In [2]:
import os
import os.path as osp
import numpy as np
import json
import torch
import re
from tqdm import tqdm
from icecream import ic
from transformers import AutoTokenizer
from summary import split_sentence
tokenizer = AutoTokenizer.from_pretrained("../models/chinese-roberta-wwm-ext")
crime_pattern = re.compile(r'已构成(.*?)罪')

In [3]:
def find_fact_idx(text):
    lead_words = ['上述指控', '']

In [6]:
def augment_edges(data_path):
    query_path = osp.join(data_path, 'query.json')
    all_candidates_path = osp.join(data_path, 'candidates')
    label_path = osp.join(data_path, 'label_top30_dict.json')
    label = json.load(open(label_path))
    edges, labels = [], []
    with open(query_path) as f:
        query_lines = f.readlines()
    num_node = 0
    new_query_inputs = []
    for query_line in tqdm(query_lines[:5]):
        query_dict = json.loads(query_line)
        query_ridx = str(query_dict['ridx'])
        query_idx = num_node
        num_node += 1
        candidates_path = osp.join(all_candidates_path, query_ridx)
        candidate_indices, augment_labels = [], []
        for candidate in os.listdir(candidates_path)[:]:
            candidate_ridx = candidate[:-5]
            candidate_idx = num_node
            num_node += 1
            candidate_indices.append(candidate_idx)
            if candidate_ridx in label[query_ridx]:
                new_label = label[query_ridx][candidate_ridx]
                augment_labels.append(new_label)
                if new_label == 3:
                    candidate_path = osp.join(candidates_path, candidate)
                    candidate_dict = json.load(open(candidate_path))
                    candidate_text = candidate_dict['ajjbqk']
                    candidate_senteces = list(split_sentence(candidate_text))
                    fact_text_idx = 0
                    for sent in candidate_senteces:
                        if '事实' in sent:
                            break
                        fact_text_idx += 1
                    if fact_text_idx < len(candidate_senteces):
                        fact_text = candidate_senteces[:fact_text_idx]
                        new_query_text = ''.join(fact_text)
                        if 200 < len(new_query_text) < 600:
                            ic(new_query_text)
            else:
                augment_labels.append(0)


train_path = '../data/origin/train'
augment_edges(train_path)


  0%|          | 0/5 [00:00<?, ?it/s]ic| new_query_text: '长沙市芙蓉区人民检察院指控：2019年6月20日晚，被告人李文涛与朋友在沁园夜宵街吃宵夜，期间喝了约二两白酒。李文涛饮酒后驾驶湘A×××＊＊小型普通客车从夜宵店出发，途经嘉雨路、营盘路。当晚23时41分许，其行驶至高家坡路农园路口路段时被执勤民警查获。现场酒精含量呼气检测结果为103毫克／100毫升。随后，民警依法将李文涛带至湖南省旺旺医院进行抽血备检。经长沙市公安局物证鉴定所鉴定，李文涛的血液中乙醇含量为103.2毫克／100毫升。公诉机关认为被告人李文涛醉酒后在道路上驾驶机动车辆，其行为构成危险驾驶罪，建议判处被告人李文涛拘役一个月，并处罚金，并建议适用缓刑。'
ic| new_query_text: '夏邑县人民检察院指控：2018年5月14日16时12分许，被告人王某某酒后驾驶豫N×××××白色小型轿车，沿夏邑县X019县道由西向东行驶至夏邑县车站镇程大庄村时，被夏邑县交警大队执勤民警当场查获。经现场对王某某进行呼气式酒精含量检测，检测结果为85mg／100ml。后抽血鉴定，王某某静脉血液中乙醇含量为102.97mg／100ml，属醉酒驾驶机动车。针对上述指控，公诉机关当庭宣读并出示了：1、被告人常住人口基本信息、驾驶人及车辆信息查询2、（2017）苏0583刑初1140号刑事判决书3、查获经过4、酒精呼吸测试单5、强制措施凭证6、当事人血样提取登记表7、商丘普济法医临床司法鉴定所血样检验报告8、被告人王某某供述等证据。'
ic| new_query_text: '公诉机关指控：2019年9月29日1时许，被告人吴巨林饮酒后驾驶湘AM8X＊＊号小型轿车从长沙市金叶神农大酒店出发，沿韶山路、五一大道、湘江大道驶入三一大道，当沿长沙市开福区三一大道由西往东行驶至银盆岭大桥东时，被开福交警大队执勤民警查获。民警现场对其进行了呼气式酒精测试，结果为155毫克／100毫升，民警将其带至湖南省中医附二医院抽取血样，经检验：被告人吴巨林血液中的乙醇含量为166.1毫克／100毫升。'
ic| new_query_text: '公诉机关指控：被告人姜柑录于2017年7月24日22时许，酒后驾驶牌号为湘F×××××小轿车沿长沙市岳麓区枫林三路由东往西行驶至岳麓区